# recount2 model building with PLIER2 and PLIER

# PLIER2

## Load libraries

In [89]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(PLIER2)
source(here("config.R"))

## Output directory

In [90]:
output_data_dir <- config$recount2$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

## Download recount2 

In [91]:
url      <- config$recount2$URL
out_dir  <- here::here("data", "recount2")
zip_file <- file.path(out_dir, "recount2.zip")
rds_file <- file.path(out_dir, "recount2_PLIER_data", "recount_data_prep_PLIER.RDS")
rpkm_file <- file.path(out_dir, "recount2_PLIER_data", "recount_rpkm.RDS")

if (!file.exists(rds_file) && !file.exists(rpkm_file)) {
    if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE)
    
    system(paste('wget', url, '-O', zip_file))
    unzip(zip_file, exdir = out_dir)
} else {
    message("Required files already exist. Skipping download and decompression.")
}

Required files already exist. Skipping download and decompression.



## Preprocess recount2 data

In [92]:
# Read preprocessed recount2 data
data_mat <- readRDS(rds_file)

meta <- list()
meta$gene_symbols <- rownames(data_mat$rpkm.cm)
meta$samples <- colnames(data_mat$rpkm.cm)

rm(data_mat)

In [93]:
# Read in recount2 file

rpkm.df <- readRDS(rpkm_file)

mart <- biomaRt::useDataset("hsapiens_gene_ensembl",
                            biomaRt::useMart("ensembl"))

genes <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"), `[[`, 1))

rpkm.df$ensembl_gene_id <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"),
                                         `[[`, 1))
                                         
gene.df <- biomaRt::getBM(filters = "ensembl_gene_id",
                          attributes = c("ensembl_gene_id", "hgnc_symbol"),
                          values = genes,
                          mart = mart)

gene.df <- gene.df %>% dplyr::filter(complete.cases(.))

rpkm.df <- dplyr::inner_join(gene.df, rpkm.df,
                             by = "ensembl_gene_id")

rownames(rpkm.df) <- make.names(rpkm.df$hgnc_symbol, unique = TRUE)

rpkm.df <- rpkm.df %>% dplyr::select(-c(ensembl_gene_id:ENSG))

data_mat <- rpkm.df[meta$gene_symbols, meta$samples]

rm(rpkm.df)

data_mat <- as.matrix(as.data.table(data_mat))

Warning message in dplyr::inner_join(gene.df, rpkm.df, by = "ensembl_gene_id"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 23 of `x` matches multiple rows in `y`.
ℹ Row 29567 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [94]:
# Create the FBM
n_genes <- length(meta$gene_symbols)
n_samples <- length(meta$samples)

fbm_file <- file.path(output_data_dir, "FBMrecount2")
recount2FBM <- FBM(nrow = n_genes, ncol = n_samples, backingfile = fbm_file, create_bk = T)

In [99]:
# Populate it with data
block_size <- config$GENERAL$CHUNK_SIZE
n_blocks <- ceiling(n_genes / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  
  recount2FBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

In [102]:
# Preprocess and z‑score FBM
prep_recount2 <- preprocessPLIER2FBM(
  fbm        = recount2FBM,
  mean_cutoff= config$recount2$GENES_MEAN_CUTOFF,
  var_cutoff = config$recount2$GENES_VAR_CUTOFF
)

recount2_fbm_filt <- prep_recount2$fbm_filtered
recount2_rowStats <- prep_recount2$rowStats
zscorePLIER2FBM(recount2_fbm_filt, recount2_rowStats)

Applying log2 transformation

Filling NAs with 0

Applying Z-score transformation



## SVD computation and SVD K estimation

In [103]:
g_fb <- nrow(recount2_fbm_filt)
samples_fb <- ncol(recount2_fbm_filt)
SVD_K_recount2  <- min(g_fb, samples_fb) - 1

message("Using SVD K = ", SVD_K_recount2)

set.seed(1)
recount2_svdRes <- big_randomSVD(recount2_fbm_filt, k = SVD_K_recount2)

Using SVD K = 6001



## Estimate K for PLIER

In [104]:
PLIER_K_recount2 <- num.pc(list(d = recount2_svdRes$d))
message("Inferred PLIER K = ", PLIER_K_recount2)

Inferred PLIER K = 129



## PLIERbase initialization

In [105]:
recount2_baseRes <- PLIERbase(
  Y      = recount2_fbm_filt,
  k      = PLIER_K_recount2,
  svdres = recount2_svdRes,
  trace  = TRUE
)

****



[1] "L1 is set to 96.4556873352599"
[1] "L2 is set to 289.36706200578"
Progress 48 / 200 | Bdiff=0.000489, minCor=0.997171Converged at iteration= 48 | Bdiff=0.000489,  tol=0.000500     

## Prepare pathway priors

In [ ]:
recount2_gmtList <- list(
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023"),
  BP            = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  MSigDB        = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=MSigDB_Hallmark_2020")
)

recount2_genes <- meta$gene_symbols[prep_recount2$kept_rows]

recount2_pathMat <- gmtListToSparseMat(recount2_gmtList)
recount2_matched <- getMatchedPathwayMat(recount2_pathMat, recount2_genes)
recount2_chatObj <- getChat(recount2_matched)

## PLIERfull

In [114]:
recount2_fullRes <- PLIERfull(
  Y                 = recount2_fbm_filt,
  priorMat          = as.matrix(recount2_matched),
  svdres            = recount2_svdRes,
  plier.base.result = recount2_baseRes,
  Chat              = recount2_chatObj,
  k                 = PLIER_K_recount2,
  doCrossval        = TRUE,
  trace             = TRUE,
  max.U.updates     = 50
)

**PLIER v2 **



using provided PLIERbase result

L1=96.4556873352599; L2=289.36706200578



Converged at 108 / 350 | Bdiff=0.000011, minCor=0.999752olumns is 98, Number of annotated columns is 98, Number of annotated columns is 99, Number of annotated columns is 100, Number of annotated columns is 100, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 97, Number of annotated columns is 93, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 94, Number of annotated columns is 95, Number of annotated columns is 96, Number of annotated columns is 97, Number of annotated columns is 97, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 95, Number of annotated columns is 96, Number of annotated columns is 93, Number of annotated columns is 93, Number of annotate

Updating Z for CV

crossValidation

There are 68  LVs with AUC>0.70

There are 32  LVs with AUC>0.90



In [115]:
saveRDS(recount2_fullRes, file = file.path(output_data_dir, "recount2_PLIER2.rds"))

In [121]:
recount2_fullRes <- readRDS(file.path(output_data_dir, "recount2_PLIER2.rds"))

In [124]:
head(recount2_fullRes$B)
dim(recount2_fullRes$B)

LV1,0.0007205184,0.01647058,0.03814555,0.01197512,0.04704399,0.04437839,0.03381749,0.03636021,0.04043869,0.03289993,⋯,-0.01736262,-0.01926078,-0.01523868,-0.01591463,-0.01855265,-0.01790466,-0.022153816,-0.01976996,-0.01984811,-0.01888447
LV2,-0.4923648979,-0.51339038,-0.50119356,-0.49921384,-0.50932023,-0.50351341,-0.39918583,-0.40266711,-0.37822289,-0.39252230,⋯,0.16563092,0.23026527,0.15791311,0.18210354,0.15538578,0.12538393,0.139928490,0.17225534,0.15470415,0.15994225
LV3,-0.0757368499,-0.06648099,-0.07406526,-0.07713371,-0.05966153,-0.06552433,-0.08328227,-0.07995257,-0.08514836,-0.07769106,⋯,0.19676653,0.18538052,0.17310187,0.20670697,0.19265481,0.15479063,0.151297105,0.21425408,0.20251804,0.14492561
LV4,-0.1961513568,-0.20566352,-0.21201147,-0.19850929,-0.19637737,-0.20395197,-0.19755121,-0.20649948,-0.19993840,-0.20461730,⋯,0.09811792,0.07008986,0.14089353,0.11110449,0.12916542,0.12924996,0.148189089,0.12431945,0.13712653,0.19727684
LV5,-0.3040506017,-0.30346150,-0.29528169,-0.29639232,-0.29417941,-0.30123343,-0.25735167,-0.26807332,-0.27006629,-0.27332290,⋯,0.13220680,0.08555006,0.07867233,0.14480664,0.11118991,0.11045196,0.101744601,0.14507086,0.21723407,0.11746733
LV6,-0.3519748697,-0.34103480,-0.32978590,-0.32507766,-0.32607388,-0.33723284,-0.31148042,-0.31001993,-0.32232314,-0.30902314,⋯,0.07010893,0.05144226,0.06915299,0.04361540,0.02826991,0.06474227,0.002771948,0.03921632,0.07130628,-0.02656028


[1]   129 37032

In [126]:
colnames(recount2_fullRes$B) <- meta$samples

In [127]:
head(recount2_fullRes$B)
dim(recount2_fullRes$B)

,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,⋯,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
LV1,0.0007205184,0.01647058,0.03814555,0.01197512,0.04704399,0.04437839,0.03381749,0.03636021,0.04043869,0.03289993,⋯,-0.01736262,-0.01926078,-0.01523868,-0.01591463,-0.01855265,-0.01790466,-0.022153816,-0.01976996,-0.01984811,-0.01888447
LV2,-0.4923648979,-0.51339038,-0.50119356,-0.49921384,-0.50932023,-0.50351341,-0.39918583,-0.40266711,-0.37822289,-0.39252230,⋯,0.16563092,0.23026527,0.15791311,0.18210354,0.15538578,0.12538393,0.139928490,0.17225534,0.15470415,0.15994225
LV3,-0.0757368499,-0.06648099,-0.07406526,-0.07713371,-0.05966153,-0.06552433,-0.08328227,-0.07995257,-0.08514836,-0.07769106,⋯,0.19676653,0.18538052,0.17310187,0.20670697,0.19265481,0.15479063,0.151297105,0.21425408,0.20251804,0.14492561
LV4,-0.1961513568,-0.20566352,-0.21201147,-0.19850929,-0.19637737,-0.20395197,-0.19755121,-0.20649948,-0.19993840,-0.20461730,⋯,0.09811792,0.07008986,0.14089353,0.11110449,0.12916542,0.12924996,0.148189089,0.12431945,0.13712653,0.19727684
LV5,-0.3040506017,-0.30346150,-0.29528169,-0.29639232,-0.29417941,-0.30123343,-0.25735167,-0.26807332,-0.27006629,-0.27332290,⋯,0.13220680,0.08555006,0.07867233,0.14480664,0.11118991,0.11045196,0.101744601,0.14507086,0.21723407,0.11746733
LV6,-0.3519748697,-0.34103480,-0.32978590,-0.32507766,-0.32607388,-0.33723284,-0.31148042,-0.31001993,-0.32232314,-0.30902314,⋯,0.07010893,0.05144226,0.06915299,0.04361540,0.02826991,0.06474227,0.002771948,0.03921632,0.07130628,-0.02656028


[1]   129 37032

In [128]:
head(recount2_fullRes$Z)
dim(recount2_fullRes$Z)

,LV 1,LV 2,LV 3,LV 4,LV 5,LV 6,LV 7,LV 8,LV 9,LV 10,⋯,LV 120,LV 121,LV 122,LV 123,LV 124,LV 125,LV 126,LV 127,LV 128,LV 129
GAS6,0,0.0000000,0,1.0096098,0.0000000,0.0000000,0.0000000,0,0.0000000,0,⋯,1.6368284,0,0,0.0000000,0.0000000,0.0000000,0,0,0,0
MMP14,0,0.0000000,0,1.3739073,0.7393090,0.0000000,0.0000000,0,0.0000000,0,⋯,0.9196013,0,0,0.0000000,0.0000000,0.5633084,0,0,0,0
DSP,0,0.6120175,0,0.0000000,0.0000000,0.4647219,0.7366880,0,0.0000000,0,⋯,0.0000000,0,0,0.0000000,0.4429041,0.0000000,0,0,0,0
MARCKSL1,0,1.1675652,0,0.4202976,0.0000000,0.7944830,0.6633716,0,0.4454415,0,⋯,0.0000000,0,0,0.6822762,0.0000000,0.0000000,0,0,0,0
SPARC,0,0.5799174,0,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,0,⋯,0.5621682,0,0,0.4044555,0.0000000,0.0000000,0,0,0,0
CTSD,0,0.0000000,0,1.1175006,0.4445312,0.0000000,0.0000000,0,0.5188999,0,⋯,0.0000000,0,0,0.0000000,0.0000000,0.0000000,0,0,0,0


[1] 6002  129

In [129]:
colnames(recount2_fullRes$Z) <- paste0('LV', seq_len(ncol(recount2_fullRes$Z)))

In [130]:
head(recount2_fullRes$Z)
dim(recount2_fullRes$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV120,LV121,LV122,LV123,LV124,LV125,LV126,LV127,LV128,LV129
GAS6,0,0.0000000,0,1.0096098,0.0000000,0.0000000,0.0000000,0,0.0000000,0,⋯,1.6368284,0,0,0.0000000,0.0000000,0.0000000,0,0,0,0
MMP14,0,0.0000000,0,1.3739073,0.7393090,0.0000000,0.0000000,0,0.0000000,0,⋯,0.9196013,0,0,0.0000000,0.0000000,0.5633084,0,0,0,0
DSP,0,0.6120175,0,0.0000000,0.0000000,0.4647219,0.7366880,0,0.0000000,0,⋯,0.0000000,0,0,0.0000000,0.4429041,0.0000000,0,0,0,0
MARCKSL1,0,1.1675652,0,0.4202976,0.0000000,0.7944830,0.6633716,0,0.4454415,0,⋯,0.0000000,0,0,0.6822762,0.0000000,0.0000000,0,0,0,0
SPARC,0,0.5799174,0,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,0,⋯,0.5621682,0,0,0.4044555,0.0000000,0.0000000,0,0,0,0
CTSD,0,0.0000000,0,1.1175006,0.4445312,0.0000000,0.0000000,0,0.5188999,0,⋯,0.0000000,0,0,0.0000000,0.0000000,0.0000000,0,0,0,0


[1] 6002  129

In [132]:
head(recount2_fullRes$summary)

,pathway,LV index,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Mitochondrial ATP Synthesis Coupled Electron Transport (GO:0042775),2,0.7954083,0.0003209802,0.002718506
2,Multivesicular Body Assembly (GO:0036258),2,0.5099627,0.4691048697,0.644630864
3,Negative Regulation of Telomere Maintenance via Telomerase (GO:0032211),2,0.6593121,0.1680680238,0.332134428
4,"Positive Regulation of mRNA Splicing, via Spliceosome (GO:0048026)",2,0.9189579,0.0194334183,0.069141030
5,Positive Regulation of Bone Mineralization (GO:0030501),3,0.2998717,0.9273754634,0.957365217
6,Positive Regulation of Production of Molecular Mediator of Immune Response (GO:0002702),3,0.4965612,0.5112146918,0.679981080


In [133]:
recount2_fullRes$summary <- recount2_fullRes$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [135]:
head(recount2_fullRes$summary)

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Mitochondrial ATP Synthesis Coupled Electron Transport (GO:0042775),LV2,0.7954083,0.0003209802,0.002718506
2,Multivesicular Body Assembly (GO:0036258),LV2,0.5099627,0.4691048697,0.644630864
3,Negative Regulation of Telomere Maintenance via Telomerase (GO:0032211),LV2,0.6593121,0.1680680238,0.332134428
4,"Positive Regulation of mRNA Splicing, via Spliceosome (GO:0048026)",LV2,0.9189579,0.0194334183,0.069141030
5,Positive Regulation of Bone Mineralization (GO:0030501),LV3,0.2998717,0.9273754634,0.957365217
6,Positive Regulation of Production of Molecular Mediator of Immune Response (GO:0002702),LV3,0.4965612,0.5112146918,0.679981080


In [ ]:
recount2_fullRes$summary  %>% 
dplyr::filter(FDR < 0.05) %>% 
dplyr::filter(AUC > 0.7)  %>% 
head()

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Mitochondrial ATP Synthesis Coupled Electron Transport (GO:0042775),LV2,0.7954083,3.209802e-04,0.002718506
2,Positive Regulation of Tumor Necrosis Factor Superfamily Cytokine Production (GO:1903557),LV3,0.7292415,1.946084e-03,0.012054101
3,Regulation of Interleukin-2 Production (GO:0032663),LV3,0.9254063,7.536747e-05,0.001075903
4,Regulation of Interleukin-8 Production (GO:0032677),LV3,0.7557399,1.640158e-03,0.010635397
5,Stomach Female 20-29 Up,LV7,0.9858300,6.004389e-03,0.030388064
6,Colon - Transverse Male 30-39 Up,LV7,0.9919872,1.611349e-04,0.001759763


In [139]:
saveRDS(recount2_fullRes, file = file.path(output_data_dir, "recount2_PLIER2.rds"))

# PLIER

Run PLIER with the same inputs than PLIER2


In [154]:
recount2_plier = PLIER::PLIER(
    recount2_fbm_filt[], 
    as.matrix(recount2_matched), 
    svdres = recount2_svdRes, 
    Chat = as.matrix(recount2_chatObj), 
    doCrossval = TRUE, 
    max.iter = 350,
    k = PLIER_K_recount2
  )

Removing 0 pathways with too few genes



[1] 289.3671
[1] "L2 is set to 289.36706200578"
[1] "L1 is set to 144.68353100289"


errorY (SVD based:best possible) = 0.8092

New L3 is 0.00218749111818289

New L3 is 0.00193045413622771

New L3 is 0.00170361979580257

New L3 is 0.00193045413622771

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00218749111818289

New L3 is 0.00193045413622771

New L3 is 0.00218749111818289

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 234 Bdiff is not decreasing

There are 53  LVs with AUC>0.70



In [155]:
head(recount2_plier$B)

LV 1,3.038562e-11,-1.164697e-13,1.499369e-11,2.420048e-11,1.076099e-11,2.400729e-11,2.450469e-11,2.529211e-11,2.492415e-11,2.411303e-11,⋯,-2.252487e-14,-9.868426e-13,7.459374e-13,-1.238694e-13,-5.471264e-13,3.208637e-13,-1.203242e-12,-9.133111e-13,-7.087071e-13,-7.032934e-13
"2,Cells - Cultured Fibroblasts Female 70-79 Up",-3.281172e-02,-3.819620e-02,-4.021169e-02,-4.102633e-02,-3.925897e-02,-3.957653e-02,-9.111154e-03,-7.944571e-03,-1.111526e-02,-5.022869e-03,⋯,3.445032e-02,9.405888e-02,5.244139e-02,9.127825e-02,8.897180e-02,6.534399e-02,7.464888e-02,1.037498e-01,4.380141e-02,9.150614e-02
"3,mRNA Splicing, via Spliceosome (GO:0000398)",-9.481710e-02,-7.740678e-02,-9.260198e-02,-8.277387e-02,-7.081893e-02,-8.502113e-02,-6.194781e-02,-6.563719e-02,-6.688297e-02,-7.041583e-02,⋯,-5.538873e-02,-6.185108e-02,-3.681471e-02,-3.960329e-02,-6.117308e-02,-2.970421e-02,-8.491269e-02,-3.287660e-02,-2.841848e-02,-1.169766e-01
"4,Translation (GO:0006412)",-1.167416e-01,-1.169561e-01,-1.241410e-01,-1.144214e-01,-1.081250e-01,-1.183666e-01,-1.348015e-01,-1.444428e-01,-1.385527e-01,-1.426251e-01,⋯,5.650564e-02,2.639246e-02,1.109185e-01,6.937041e-02,8.791064e-02,1.055854e-01,1.084018e-01,7.893540e-02,7.554681e-02,1.749875e-01
"5,Double-Strand Break Repair via Homologous Recombination (GO:0000724)",-1.475776e-02,-2.136897e-02,-2.183746e-02,-1.868801e-02,-1.880394e-02,-1.911732e-02,-2.733048e-02,-3.572729e-02,-3.150585e-02,-2.717683e-02,⋯,4.954903e-03,2.586807e-02,2.185875e-02,4.275850e-02,4.369622e-02,5.298591e-02,-1.144091e-02,5.217841e-02,-1.841314e-02,2.890525e-02
LV 6,1.137411e-02,1.100171e-02,2.021871e-02,2.901634e-02,1.232140e-02,1.205028e-02,9.152861e-03,1.060361e-02,-6.413729e-04,6.352040e-03,⋯,-7.302503e-02,-5.922457e-02,-7.198972e-02,-5.402172e-02,-6.231897e-02,-6.095302e-02,-5.806830e-02,-6.060223e-02,-7.126057e-02,-5.891329e-02


In [143]:
colnames(recount2_plier$Z) <- paste0('LV', seq_len(ncol(recount2_fullRes$Z)))

In [144]:
head(recount2_plier$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV120,LV121,LV122,LV123,LV124,LV125,LV126,LV127,LV128,LV129
GAS6,0,0.13030280,0.0000000,1.0117825,0.06469198,0.62891552,0.05223254,0.0000000000,0.0000000,0.05991932,⋯,0.3951229,0.04813642,0.25652596,0.0000000000,0.04389765,0.000000000,0.000000000,0.17037143,0.46255746,0.0000000
MMP14,0,0.03662275,0.0000000,1.1923262,0.00000000,0.33187112,0.19684278,0.0005136506,0.0000000,0.00000000,⋯,0.0236564,0.00000000,0.02468694,0.0000000000,0.00000000,0.000000000,0.004685637,0.00000000,0.53684948,0.0000000
DSP,0,0.13073012,0.0000000,0.0000000,0.00000000,0.04233393,0.59420422,0.0000000000,0.0000000,0.14368568,⋯,0.2656747,0.00000000,0.00000000,0.0009715591,0.00000000,0.000000000,0.000000000,0.00000000,0.17355439,0.0000000
MARCKSL1,0,0.04233664,0.2503717,0.4627274,0.00000000,0.00000000,0.59996857,0.1305426424,0.4276414,0.06021881,⋯,0.8707375,0.03480199,0.00000000,1.4058449268,0.01264275,0.461875616,0.000000000,0.00000000,0.01256068,0.0000000
SPARC,0,0.00000000,0.0000000,0.1280858,0.00000000,0.00000000,0.00000000,0.0000000000,0.1900638,0.00000000,⋯,0.0000000,0.00000000,0.13795806,0.5095597717,0.00000000,0.008475446,0.000000000,0.55539274,0.08980203,1.3738277
CTSD,0,0.00000000,0.2271552,1.1253868,0.00000000,0.10498905,0.20927556,0.0066196885,0.4120490,0.00000000,⋯,0.6490549,0.00000000,0.00000000,0.1841587986,0.00000000,0.000000000,0.132311549,0.02588671,0.00000000,0.2489984


In [ ]:
recount2_plier$summary <- recount2_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [ ]:
head(recount2_plier$summary)

In [ ]:
recount2_plier$summary  %>% 
dplyr::filter(FDR < 0.05) %>% 
dplyr::filter(AUC > 0.7)  %>% 
head()

In [140]:
saveRDS(recount2_plier, file = file.path(output_data_dir, "recount2_PLIER.rds"))